<a href="https://colab.research.google.com/github/dnth/zenml-optuna/blob/master/pytorch_hpo_fmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install zenml[server] optuna optuna-dashboard plotly

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True)

In [ ]:
!rm -rf .zen
!zenml init

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim


from zenml.pipelines import pipeline
from zenml.steps import step, Output, BaseParameters

import optuna
from optuna.trial import TrialState
from optuna.storages import RetryFailedTrialCallback

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
@pipeline
def pytorch_hpo_pipeline(
    load_data,
    run_hpo,
    train_test,
):
    """A `pipeline` to load data, load model, and train/evaluate the model."""
    train_dataloader, test_dataloader = load_data()
    best_hparams = run_hpo(train_dataloader, test_dataloader)
    train_test(best_hparams, train_dataloader, test_dataloader)


In [3]:
@step
def load_data() -> Output(
    train_dataloader=DataLoader, test_dataloader=DataLoader
):
    """A `step` to load the Fashion MNIST dataset as a tuple of torch Datasets."""

    batch_size = 64

    # Download training data from open datasets.
    training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    # Download test data from open datasets.
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader

In [4]:
def define_hpo_model(trial):
    CLASSES = 10
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []
    in_features = 28 * 28
    layers.append(nn.Flatten())
    
    for i in range(n_layers):
        out_features = trial.suggest_int(f"n_units_l{i}", 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float(f"dropout_l{i}", 0.2, 0.5)
        layers.append(nn.Dropout(p))
        in_features = out_features
        
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

def objective(trial, train_dataloader, test_dataloader):

    # Generate model
    model = define_hpo_model(trial)
    
    # Train model
    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer_name = trial.suggest_categorical("optimizer_name", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    size = len(train_dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    test_accuracy = 100*correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return test_accuracy


class OptunaStudyParams(BaseParameters):
    """Optuna Study Params"""
    n_trials: int
    timeout: int
    direction: str
    study_name: str
    sqlite_db_name: str

@step
def run_hpo(
    study_params: OptunaStudyParams,
    train_dataloader: DataLoader, 
    test_dataloader: DataLoader
) -> dict:
    
    storage = optuna.storages.RDBStorage(
        study_params.sqlite_db_name,
        heartbeat_interval=1,
        failed_trial_callback=RetryFailedTrialCallback(),
    )

    study = optuna.create_study(study_name=study_params.study_name, direction=study_params.direction, storage=storage)
    study.optimize(lambda trial: objective(trial, train_dataloader, test_dataloader), n_trials=study_params.n_trials, timeout=study_params.timeout)
    
    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    best = trial.params

    print("Found best hyperparam dict from optimization work!")
    print(best)
    
    return best

In [5]:
def define_model(hparam:dict):
    CLASSES = 10
    layers = []
    in_features = 28 * 28
    layers.append(nn.Flatten())
    
    print(hparam)
    
    for i in range(hparam['n_layers']):
        out_features = hparam[f"n_units_l{i}"]
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = hparam[f"dropout_l{i}"]
        layers.append(nn.Dropout(p))
        in_features = out_features
        
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    """A function to train a model for one epoch."""
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    """A function to test a model on the validation / test dataset."""
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    test_accuracy = 100*correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return test_accuracy

In [7]:
@step
def train_test(
    best_hparams: dict,
    train_dataloader: DataLoader, 
    test_dataloader: DataLoader
) -> Output(trained_model=nn.Module, test_acc=float):
    """A `step` to train and evaluate a torch model on given dataloaders."""
    
    epochs = 5
    
    model = define_model(best_hparams)
    
    print("Constructing optimized model:")
    print(model)

    print("Using optimized hyperparams")
    print(best_hparams)

    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    
    lr = best_hparams['lr']
    optimizer_name = best_hparams['optimizer_name']
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    test_acc = 0
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test_acc = test(test_dataloader, model, loss_fn)
    print("Done!")

    return model, test_acc

In [8]:
pytorch_hpo_pipeline(
    load_data=load_data(),
    run_hpo=run_hpo(study_params=OptunaStudyParams(n_trials=3, timeout=300, direction='maximize', 
                                                   study_name="zenml-optuna-hpo", 
                                                   sqlite_db_name="sqlite:///zenml-optuna.db")),
    train_test=train_test(),
).run(unlisted=True)

Unable to find ZenML repository in your current working directory (/home/dnth/Desktop/zenml-optuna) or any parent directories. If you want to use an existing repository which is in a different location, set the environment variable 'ZENML_REPOSITORY_PATH'. If you want to create a new repository, run zenml init.
Running without an active repository root.
Running unlisted pipeline on stack default (caching enabled)
Step load_data has started.
Using cached version of load_data.
Step load_data has finished in 0.014s.
Step run_hpo has started.


/tmp/ipykernel_27147/4234398789.py:84: ExperimentalWarning: RetryFailedTrialCallback is experimental (supported from v2.8.0). The interface can change in the future.
  failed_trial_callback=RetryFailedTrialCallback(),
[I 2022-11-29 13:14:16,299] A new study created in RDB with name: zenml-optuna-hpo
/home/dnth/anaconda3/envs/zenml-hpo/lib/python3.8/site-packages/optuna/study/_optimize.py:185: ExperimentalWarning: fail_stale_trials is experimental (supported from v2.9.0). The interface can change in the future.
  optuna.storages.fail_stale_trials(study)
[I 2022-11-29 13:14:22,798] Trial 0 finished with value: 80.46 and parameters: {'n_layers': 2, 'n_units_l0': 127, 'dropout_l0': 0.31505943365121664, 'n_units_l1': 35, 'dropout_l1': 0.33565787741191566, 'optimizer_name': 'Adam', 'lr': 0.00043412763827912386}. Best is trial 0 with value: 80.46.


Test Error: 
 Accuracy: 80.5%, Avg loss: 0.533421 



[I 2022-11-29 13:14:27,595] Trial 1 finished with value: 51.160000000000004 and parameters: {'n_layers': 2, 'n_units_l0': 68, 'dropout_l0': 0.41756314257063504, 'n_units_l1': 79, 'dropout_l1': 0.24520208352048595, 'optimizer_name': 'Adam', 'lr': 1.353525056572201e-05}. Best is trial 0 with value: 80.46.


Test Error: 
 Accuracy: 51.2%, Avg loss: 1.793898 



[I 2022-11-29 13:14:32,327] Trial 2 finished with value: 70.64 and parameters: {'n_layers': 2, 'n_units_l0': 60, 'dropout_l0': 0.42722587596314654, 'n_units_l1': 24, 'dropout_l1': 0.27925380754814666, 'optimizer_name': 'RMSprop', 'lr': 0.016292340983987012}. Best is trial 0 with value: 80.46.


Test Error: 
 Accuracy: 70.6%, Avg loss: 0.727914 

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  80.46
  Params: 
    dropout_l0: 0.31505943365121664
    dropout_l1: 0.33565787741191566
    lr: 0.00043412763827912386
    n_layers: 2
    n_units_l0: 127
    n_units_l1: 35
    optimizer_name: Adam
Found best hyperparam dict from optimization work!
{'dropout_l0': 0.31505943365121664, 'dropout_l1': 0.33565787741191566, 'lr': 0.00043412763827912386, 'n_layers': 2, 'n_units_l0': 127, 'n_units_l1': 35, 'optimizer_name': 'Adam'}
Step run_hpo has finished in 16.159s.
Step train_test has started.
{'dropout_l0': 0.31505943365121664, 'dropout_l1': 0.33565787741191566, 'lr': 0.00043412763827912386, 'n_layers': 2, 'n_units_l0': 127, 'n_units_l1': 35, 'optimizer_name': 'Adam'}
Constructing optimized model:
Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=127, bias=T

In [10]:
import optuna
from optuna.storages import RetryFailedTrialCallback

storage = optuna.storages.RDBStorage(
        "sqlite:///zenml-optuna.db",
        heartbeat_interval=1,
        failed_trial_callback=RetryFailedTrialCallback(),
    )

study = optuna.load_study(study_name='zenml-optuna-hpo',storage=storage)

/tmp/ipykernel_27147/2206632645.py:7: ExperimentalWarning: RetryFailedTrialCallback is experimental (supported from v2.8.0). The interface can change in the future.
  failed_trial_callback=RetryFailedTrialCallback(),


In [11]:
#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [12]:
# Visualize the optimization history. See :func:`~optuna.visualization.plot_optimization_history` for the details.
plot_optimization_history(study)

In [13]:
# Visualize high-dimensional parameter relationships. See :func:`~optuna.visualization.plot_parallel_coordinate` for the details.
plot_parallel_coordinate(study)

In [14]:
# Visualize individual hyperparameters as slice plot. See :func:`~optuna.visualization.plot_slice` for the details.
plot_slice(study)

In [15]:
# Visualize parameter importances. See :func:`~optuna.visualization.plot_param_importances` for the details.
#In this case, we have only one parameter.
plot_param_importances(study)

In [16]:
# Visualize empirical distribution function. See :func:`~optuna.visualization.plot_edf` for the details.
plot_edf(study)